In [ ]:
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments, Trainer, pipeline
from peft import LoraConfig
from datasets import Dataset
import datasets
from trl import SFTTrainer, PPOTrainer

from tqdm import tqdm
#load model name
# model_name = "qwen/Qwen2.5-0.5B"
import argparse

/home/intern/miniconda3/envs/hoang/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = 'qwen/Qwen2.5-1.5B'
model_pretrained_path = 'cp1'

In [3]:
based_model = AutoModelForCausalLM.from_pretrained(model_name,
                                                   low_cpu_mem_usage=True,
    # quantization_config=quant_config,
    # torch_dtype=torch.float16,
    device_map={"":0}
)

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [5]:
based_model.from_pretrained(model_pretrained_path)

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): lora.Linear(
            (base_layer): Linear(in_features=1536, out_features=1536, bias=True)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.1, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=1536, out_features=128, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=128, out_features=1536, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): lora.Linear(
            (base_layer): Linear(in_features=1536, out_features=25

In [ ]:
# based_model = AutoModelForCausalLM.from_pretrained(model_pretrained_path)

In [6]:
question = "A merchant wants to make a choice of purchase between 2 purchase plans: jewelry worth $5,000 or electronic gadgets worth $8,000. His financial advisor speculates that the jewelry market will go up 2.5% while the electronic gadgets market will rise 1.2% within the same month. If the merchant is looking to maximize profit at the end of this month by making a choice, how much profit would this be?"
prompt1 = '''<|im_start|>user
{question}<|im_end|>

<|im_start|>assistant'''

input1 = prompt1.format(question = question)
print(input1)

<|im_start|>user
A merchant wants to make a choice of purchase between 2 purchase plans: jewelry worth $5,000 or electronic gadgets worth $8,000. His financial advisor speculates that the jewelry market will go up 2.5% while the electronic gadgets market will rise 1.2% within the same month. If the merchant is looking to maximize profit at the end of this month by making a choice, how much profit would this be?<|im_end|>

<|im_start|>assistant


# hard test 2

In [7]:
question = "John takes care of 10 dogs. Each dog takes .5 hours a day to walk and take care of their business. How many hours a week does he spend taking care of dogs?"
prompt2 = '''<|im_start|>user
{question}<|im_end|>

<|im_start|>assistant'''

input2 = prompt2.format(question = question)
print(input2)

<|im_start|>user
John takes care of 10 dogs. Each dog takes .5 hours a day to walk and take care of their business. How many hours a week does he spend taking care of dogs?<|im_end|>

<|im_start|>assistant


# Run part

In [8]:
output = based_model.generate(tokenizer(input1, return_tensors='pt').input_ids.to('cuda'), max_new_tokens = 300, eos_token_id = [151643, 151645])
print(tokenizer.decode(output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|im_start|>user
A merchant wants to make a choice of purchase between 2 purchase plans: jewelry worth $5,000 or electronic gadgets worth $8,000. His financial advisor speculates that the jewelry market will go up 2.5% while the electronic gadgets market will rise 1.2% within the same month. If the merchant is looking to maximize profit at the end of this month by making a choice, how much profit would this be?<|im_end|>

<|im_start|>assistant
The profit from the jewelry purchase would be $5,000 * 2.5% = $125. The profit from the electronic gadgets purchase would be $8,000 * 1.2% = $96. The total profit would be $125 + $96 = $221.

imdi
imdiassistant
The profit from the jewelry purchase would be $5,000 * 2.5% = $125. The profit from the electronic gadgets purchase would be $8,000 * 1.2% = $96. The total profit would be $125 + $96 = $221.

imdi
imdiassistant
The profit from the jewelry purchase would be $5,000 * 2.5% = $125. The profit from the electronic gadgets purchase would be $8,00

In [9]:
output = based_model.generate(tokenizer(input2, return_tensors='pt').input_ids.to('cuda'), max_new_tokens = 300, eos_token_id = [151643, 151645])
print(tokenizer.decode(output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<|im_start|>user
John takes care of 10 dogs. Each dog takes .5 hours a day to walk and take care of their business. How many hours a week does he spend taking care of dogs?<|im_end|>

<|im_start|>assistant
John takes care of 10 dogs. Each dog takes .5 hours a day to walk and take care of their business. How many hours a week does he spend taking care of dogs?界第一行

界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
界第一行
